In [3]:
import pandas as pd
import numpy as np
import torch    
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


In [10]:
data = pd.read_csv(r"D:\temp\Car_Insurace_Claim\Claim12.csv", encoding='latin1')


C:\Users\VIP\AppData\Local\Temp\ipykernel_23796\148078192.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r"D:\temp\Car_Insurace_Claim\Claim12.csv", encoding='latin1')


In [19]:
pd.set_option('display.max_columns', None)

In [13]:
data.describe()

,vtr_dau,str_dau,gesbeitr,SDANZAHL,kosten_verw,kosten_prov,alter,SDBEITR5,altgrp,LEISTUNG_IN_KW,KILOMETERSTAND_CLEAN
count,596851.000000,194867.000000,596851.000000,596851.000000,596851.000000,596851.000000,596851.000000,596851.000000,596851.000000,536976.000000,462334.000000
mean,407.764819,226.091190,270.493693,0.075783,31.117352,27.469187,126.276727,273.373513,4.829783,97.248337,114135.447010
std,145.858061,155.064849,315.895692,0.290998,34.754165,29.299366,254.368018,277.559148,1.995687,44.858733,80737.525936
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000
25%,365.000000,91.000000,93.330000,0.000000,8.110000,6.880000,41.000000,72.980000,4.000000,68.000000,53000.000000
50%,467.000000,150.000000,210.880000,0.000000,24.570000,21.140000,55.000000,220.160000,5.000000,90.000000,102000.000000
75%,516.000000,360.000000,347.610000,0.000000,42.080000,37.610000,68.000000,377.480000,7.000000,110.000000,161000.000000
max,516.000000,516.000000,15193.570000,5.000000,1918.650000,933.900000,999.000000,11052.120000,7.000000,999.000000,999999.000000


In [14]:
data.isnull().sum()

vsnr                         0
status                       0
spartek                      0
beginn                       0
vtr_dau                      0
str_dau                 401984
kund_beg                     0
vtrweg                       0
gesbeitr                     0
SDANZAHL                     0
kosten_verw                  0
kosten_prov                  0
gfeld                        0
ort                          0
plz                          0
alter                        0
anr                          0
SDBEITR5                     0
altgrp                       0
HERSTELLER                   0
LEISTUNG_IN_KW           59875
KILOMETERSTAND_CLEAN    134517
First_reg                45610
dtype: int64

In [27]:
# Check number of null values in First_reg before dropping
print("Null values in First_reg before dropping:", data['First_reg'].isnull().sum())

# Drop rows where First_reg is null
data = data.dropna(subset=['First_reg'])

# Verify the drop operation
print("\nNull values in First_reg after dropping:", data['First_reg'].isnull().sum())
print("\nNew shape of dataset:", data.shape)

Null values in First_reg before dropping: 45610

Null values in First_reg after dropping: 0

New shape of dataset: (551241, 25)


In [28]:
# Check total number of duplicate rows
print("Total number of duplicate rows:", data.duplicated().sum())

Total number of duplicate rows: 0


In [29]:
data['beginn'] = pd.to_datetime(data['beginn'], format='%d.%m.%Y')
data['beg_year'] = data['beginn'].dt.year


In [30]:
data['kund_beg'] = pd.to_datetime(data['kund_beg'], format='%d.%m.%Y')
data['Cus_Since'] = data['kund_beg'].dt.year



In [33]:
# Convert First_reg to datetime using the correct format
data['First_reg'] = pd.to_datetime(data['First_reg'], format='%d%b%Y')

# Convert to the desired format YYYY-MM-DD
data['First_reg'] = data['First_reg'].dt.strftime('%Y-%m-%d')

# Verify the conversion
print("First few rows of First_reg after conversion:")
print(data['First_reg'].head())

First few rows of First_reg after conversion:
2     2012-04-23
3     2012-04-23
8     2018-07-23
9     2018-07-23
10    2023-03-30
Name: First_reg, dtype: object


In [36]:
# Since First_reg is already in YYYY-MM-DD format, just convert to datetime
data['First_reg'] = pd.to_datetime(data['First_reg'])

# Extract year into Car_regg column
data['Car_regg'] = data['First_reg'].dt.year

# Verify the conversion
print("First few rows after conversion:")
print(data[['First_reg', 'Car_regg']].head())

First few rows after conversion:
    First_reg  Car_regg
2  2012-04-23      2012
3  2012-04-23      2012
8  2018-07-23      2018
9  2018-07-23      2018
10 2023-03-30      2023


In [37]:
# Get today's date
today = pd.Timestamp.today()

# Calculate days and years since registration
data['Days_Since_Reg'] = (today - data['First_reg']).dt.days
data['Years_Since_Reg'] = (data['Days_Since_Reg'] / 365.25).round(2)

# Display the results
print("First few rows showing car age:")
print(data[['First_reg', 'Days_Since_Reg', 'Years_Since_Reg']].head())

# Show some basic statistics
print("\nSummary statistics of vehicle age:")
print(data[['Days_Since_Reg', 'Years_Since_Reg']].describe())

First few rows showing car age:
    First_reg  Days_Since_Reg  Years_Since_Reg
2  2012-04-23            4838            13.25
3  2012-04-23            4838            13.25
8  2018-07-23            2556             7.00
9  2018-07-23            2556             7.00
10 2023-03-30             845             2.31

Summary statistics of vehicle age:
       Days_Since_Reg  Years_Since_Reg
count   551241.000000    551241.000000
mean      4117.218601        11.272346
std       2641.990633         7.233390
min     -64500.000000      -176.590000
25%       2254.000000         6.170000
50%       3737.000000        10.230000
75%       5570.000000        15.250000
max      45858.000000       125.550000


In [40]:
data.head()

,vsnr,status,spartek,beginn,vtr_dau,str_dau,kund_beg,vtrweg,gesbeitr,SDANZAHL,kosten_verw,kosten_prov,gfeld,ort,plz,alter,anr,SDBEITR5,altgrp,HERSTELLER,LEISTUNG_IN_KW,KILOMETERSTAND_CLEAN,First_reg,beg_year,Cus_Since,Car_regg,Days_Since_Reg,Years_Since_Reg
2,01013507-009,Aktiv,KH,2024-01-23,494.0,NaN,1982-11-05,V,333.96,0,70.33,34.20,70269/ Land- und Forstwirtschaft,Husby,24975,61,71295/ Herr,405.39,6,MERCEDES-BENZ S. AUCH DAI,115.0,34500.0,2012-04-23,2024,1982,2012,4838,13.25
3,01013507-009,Aktiv,VK,2024-01-23,494.0,NaN,1982-11-05,V,676.78,0,137.09,82.85,70269/ Land- und Forstwirtschaft,Husby,24975,61,71295/ Herr,804.89,6,MERCEDES-BENZ S. AUCH DAI,115.0,34500.0,2012-04-23,2024,1982,2012,4838,13.25
8,01030924-012,Aktiv,KH,2024-01-31,486.0,NaN,1932-04-02,V,194.48,0,41.63,20.24,70269/ Land- und Forstwirtschaft,Boren,24392,64,71295/ Herr,239.93,7,VOLKSWAGEN-VW,110.0,NaN,2018-07-23,2024,1932,2018,2556,7.00
9,01030924-012,Aktiv,VK,2024-01-31,486.0,NaN,1932-04-02,V,422.20,0,84.79,51.26,70269/ Land- und Forstwirtschaft,Boren,24392,64,71295/ Herr,497.97,7,VOLKSWAGEN-VW,110.0,NaN,2018-07-23,2024,1932,2018,2556,7.00
10,01030990-018,Aktiv,KH,2024-03-20,437.0,NaN,1947-09-20,V,413.25,0,87.62,42.61,70271/ Selbständige,Seedorf,23823,63,71295/ Herr,505.02,7,TOYOTA (J),72.0,NaN,2023-03-30,2024,1947,2023,845,2.31


In [39]:
# Get today's date
today = pd.Timestamp.today()

# Calculate days and years since registration
data['Days_Since_Reg'] = (today - data['First_reg']).dt.days
data['Years_Since_Reg'] = (data['Days_Since_Reg'] / 365.25).round(2)

# Display the results
print("First few rows showing car age:")
print(data[['First_reg', 'Days_Since_Reg', 'Years_Since_Reg']].head())

# Show some basic statistics
print("\nSummary statistics of vehicle age:")
print(data[['Days_Since_Reg', 'Years_Since_Reg']].describe())

First few rows showing car age:
    First_reg  Days_Since_Reg  Years_Since_Reg
2  2012-04-23            4838            13.25
3  2012-04-23            4838            13.25
8  2018-07-23            2556             7.00
9  2018-07-23            2556             7.00
10 2023-03-30             845             2.31

Summary statistics of vehicle age:
       Days_Since_Reg  Years_Since_Reg
count   551241.000000    551241.000000
mean      4117.218601        11.272346
std       2641.990633         7.233390
min     -64500.000000      -176.590000
25%       2254.000000         6.170000
50%       3737.000000        10.230000
75%       5570.000000        15.250000
max      45858.000000       125.550000


In [41]:
# Extract everything after '/' in anr column
data['anr'] = data['anr'].str.split('/').str[1]

# Verify the changes
print("First few rows of cleaned anr column:")
print(data['anr'].head())

First few rows of cleaned anr column:
2      Herr
3      Herr
8      Herr
9      Herr
10     Herr
Name: anr, dtype: object


In [42]:
data.head()

,vsnr,status,spartek,beginn,vtr_dau,str_dau,kund_beg,vtrweg,gesbeitr,SDANZAHL,kosten_verw,kosten_prov,gfeld,ort,plz,alter,anr,SDBEITR5,altgrp,HERSTELLER,LEISTUNG_IN_KW,KILOMETERSTAND_CLEAN,First_reg,beg_year,Cus_Since,Car_regg,Days_Since_Reg,Years_Since_Reg
2,01013507-009,Aktiv,KH,2024-01-23,494.0,NaN,1982-11-05,V,333.96,0,70.33,34.20,70269/ Land- und Forstwirtschaft,Husby,24975,61,Herr,405.39,6,MERCEDES-BENZ S. AUCH DAI,115.0,34500.0,2012-04-23,2024,1982,2012,4838,13.25
3,01013507-009,Aktiv,VK,2024-01-23,494.0,NaN,1982-11-05,V,676.78,0,137.09,82.85,70269/ Land- und Forstwirtschaft,Husby,24975,61,Herr,804.89,6,MERCEDES-BENZ S. AUCH DAI,115.0,34500.0,2012-04-23,2024,1982,2012,4838,13.25
8,01030924-012,Aktiv,KH,2024-01-31,486.0,NaN,1932-04-02,V,194.48,0,41.63,20.24,70269/ Land- und Forstwirtschaft,Boren,24392,64,Herr,239.93,7,VOLKSWAGEN-VW,110.0,NaN,2018-07-23,2024,1932,2018,2556,7.00
9,01030924-012,Aktiv,VK,2024-01-31,486.0,NaN,1932-04-02,V,422.20,0,84.79,51.26,70269/ Land- und Forstwirtschaft,Boren,24392,64,Herr,497.97,7,VOLKSWAGEN-VW,110.0,NaN,2018-07-23,2024,1932,2018,2556,7.00
10,01030990-018,Aktiv,KH,2024-03-20,437.0,NaN,1947-09-20,V,413.25,0,87.62,42.61,70271/ Selbständige,Seedorf,23823,63,Herr,505.02,7,TOYOTA (J),72.0,NaN,2023-03-30,2024,1947,2023,845,2.31


In [43]:
data['KILOMETERSTAND_CLEAN'].fillna(0, inplace=True)

C:\Users\VIP\AppData\Local\Temp\ipykernel_23796\2723622540.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['KILOMETERSTAND_CLEAN'].fillna(0, inplace=True)


In [45]:
# Mapping postal codes to states in Germany
# The mapping is based on the first two digits of the postal code (PLZ)
plz_state_mapping = {
    '01': 'Sachsen',
    '02': 'Sachsen',
    '03': 'Brandenburg',
    '04': 'Sachsen',
    '05': 'Sachsen-Anhalt',
    '06': 'Sachsen-Anhalt',
    '07': 'Thüringen',
    '08': 'Sachsen',
    '09': 'Sachsen',

    '10': 'Berlin', '11': 'Berlin', '12': 'Brandenburg',
    '13': 'Brandenburg', '14': 'Brandenburg', '15': 'Brandenburg',
    '16': 'Brandenburg',

    '17': 'Mecklenburg-Vorpommern', '18': 'Mecklenburg-Vorpommern',
    '19': 'Mecklenburg-Vorpommern',

    '20': 'Schleswig-Holstein', '21': 'Schleswig-Holstein',
    '22': 'Hamburg', '23': 'Schleswig-Holstein',

    '24': 'Schleswig-Holstein', '25': 'Schleswig-Holstein',
    '26': 'Niedersachsen', '27': 'Bremen', '28': 'Bremen',
    '29': 'Niedersachsen',

    '30': 'Niedersachsen', '31': 'Niedersachsen', '32': 'Nordrhein-Westfalen',
    '33': 'Nordrhein-Westfalen', '34': 'Hessen',

    '35': 'Hessen', '36': 'Hessen', '37': 'Niedersachsen',
    '38': 'Niedersachsen', '39': 'Sachsen-Anhalt',

    '40': 'Nordrhein-Westfalen', '41': 'Nordrhein-Westfalen',
    '42': 'Nordrhein-Westfalen', '44': 'Nordrhein-Westfalen',
    '45': 'Nordrhein-Westfalen', '46': 'Nordrhein-Westfalen',

    '47': 'Nordrhein-Westfalen', '48': 'Nordrhein-Westfalen',
    '49': 'Niedersachsen',

    '50': 'Nordrhein-Westfalen', '51': 'Nordrhein-Westfalen',
    '52': 'Nordrhein-Westfalen', '53': 'Nordrhein-Westfalen',
    '54': 'Rheinland-Pfalz', '55': 'Rheinland-Pfalz',

    '56': 'Rheinland-Pfalz', '57': 'Nordrhein-Westfalen',
    '58': 'Nordrhein-Westfalen', '59': 'Nordrhein-Westfalen',

    '60': 'Hessen', '61': 'Hessen', '62': 'Hessen',
    '63': 'Hessen', '64': 'Hessen', '65': 'Hessen',

    '66': 'Saarland', '67': 'Rheinland-Pfalz', '68': 'Rheinland-Pfalz',
    '69': 'Hessen',

    '70': 'Baden-Württemberg', '71': 'Baden-Württemberg',
    '72': 'Baden-Württemberg', '73': 'Baden-Württemberg',
    '74': 'Baden-Württemberg', '75': 'Baden-Württemberg',
    '76': 'Baden-Württemberg', '77': 'Baden-Württemberg',
    '78': 'Baden-Württemberg', '79': 'Baden-Württemberg',

    '80': 'Bayern', '81': 'Bayern', '82': 'Bayern', '83': 'Bayern',
    '84': 'Bayern', '85': 'Bayern', '86': 'Bayern', '87': 'Bayern',
    '88': 'Bayern', '89': 'Bayern',

    '90': 'Bayern', '91': 'Bayern', '92': 'Bayern', '93': 'Bayern',
    '94': 'Bayern', '95': 'Bayern', '96': 'Bayern',

    '97': 'Bayern', '98': 'Thüringen', '99': 'Thüringen'
}

def get_state_from_plz(plz):
    if pd.isnull(plz):
        return None
    try:
        plz_str = str(plz).zfill(5)  # ensure it's 5 digits
        return plz_state_mapping.get(plz_str[:2])
    except:
        return None
    
data['state'] = data['plz'].apply(get_state_from_plz)
data['state_id'] = pd.factorize(data['state'])[0]+1
data = data.dropna(subset=['state'])


In [ ]:
# Extract everything after '/' in anr column
data['gfeld'] = data['gfeld'].str.split('/').str[1]

# Verify the changes
print("First few rows of cleaned anr column:")
print(data['gfeld'].head())